For undirected models, we have an unormalized probability distribution:
$$p(x;\theta) = \frac{1}{Z(\theta)} \tilde{p}(x;\theta)$$

with $Z$ the partition function:
$$Z(\theta) = \sum_x \tilde{p}(x, \theta)$$
$$Z(\theta) = \int \tilde{p}(x, \theta) dx$$

Some models are designed to have a tractable $Z$, or that we don't need $p(x)$, but other interesting models have an intractable $Z$.

# The Log-Likelihood Gradient

To train a model by maximum likelihood, we need the partition function gradient:
$$\nabla_\theta \log p(x;\theta) = \nabla_\theta \log \tilde{p}(x;\theta) - \nabla_\theta \log Z(\theta)$$  

This is the decomposition into positive phase and negative phase of learning. The negative phase is difficult for most graphical models, like RMBs.  
For models that guarantee $p(x) > 0 \forall x$, the negative phrae can be rewritten as:
$$\nabla_\theta \log Z(\theta) = \mathbb{E}_{x \sim p(x)} \nabla_\theta \log \tilde{p}(x)$$  

This is a basis for MC methods to approximately maximime the likelihood of models with intractable partition functions.  
In the positive phase we increase $\log \tilde{p}(x)$ for $x$ drawn from the training data, and in the negative phase we decrease $\log \hat{p}(x)$ with $x$ drawn from the model distribution.

# Stochastic Maximum Likelihood and Contrastice Divergence

The naive approach is to compute $\nabla_\theta \log Z$ by taking $m$ samples from the model using MCMC, at each gradient step.  
But burning markov chains at each iteration is computationally infeasible.  

The negative phase can be seen as pushing up on the model distribution where the data occurs, and pushing down where the model samples occur. Approximations to the negative phase make them cheaper, but pushing down in the wrong location.  

Contrastive divergence (CD) algorithm initialize the Markov chain with samples from the data distribution, allowing to burn in faster.  
CD is only an approximation of the correct negative phase, and fails to suppress regions of high probability far from the training data. There regions are called spurious modes.  

CD is a biased estimator for RBMs, but it can still train RBMS and other shallow models.  
CD doesn't help for deep models, it's difficult to obtain samples of the hidden units, they are not given by the dataset, and we need to burn in the hidden units.  

Persistence Contrastive Divergence (PCD) start the Markov chain at each gradient step from their states from the previous gradient state. The idea is that with small gradient steps the model from the previous step will be similar to the actual one.  
PCD is resistant to spurious modes, and provides an initialization point for both visible and hidden units, it can train deep models efficiently.  
It can become inaccurate if the gradient step move the model faster than the chain can mix between steps ($k$ too small, learning rate too large).  

CD has lower variance than the estimator based on exact sampling, because it uses the same training points for both the positive and negative phase.   

Fast PCD tries to accelerate mixing during learning by replacing the parameters $\theta$ with:
$$\theta = \theta^{(\text{slow})} + \theta^{(\text{fast})}$$  
The fast copy of the parametrs are trained with a large learning rate, allowing to push the chain to new territory, for faster mixing.  

Because MCMCM methods provide an estimate of $\nabla_\theta \log Z$, we can use any other method to estimate the positive phase $\nabla_\theta \log \tilde{p}(x)$, for example methods that provides a lower boun on $\tilde{p}$.

# Pseudolikelihood

Computing ratios of probabilities avoid computing $Z$. Suppose we parition $x$ into $a, b, c$, we get:
$$p(a|b) = \frac{p(a,b)}{\sum_{a,c} p(a,b,c)} = \frac{\tilde{p}(a,b)}{\sum_{a,c} \tilde{p}(a,b,c)}$$

In the extreme case where $a$ contains one variable and $c$ is empty, we only have to marginalize over one variable.  
This happens if me replace the log likelihood by the pseudolikelihood:
$$\sum_{i=1}^n \log p(x_i|x_{-i})$$

If each random variable has $k$ diffent values, we need $k*n$ evalutations of $\tilde{p}$ instead of $k^n$ for $Z$.  
Estimation by maxiziming the pseudolikelihood is asymptotically consistent.  

Generalizd pseudolikelihood is a tradeoff between computational complexity and deviation from maximuum likelihood:
$$\sum_{i=1}^m \log p(x_{S^{(i)}}|X_{-S^{(i)}})$$

with $S^{(i)}$ set of indices for variables of $x$.  

Pseudolikelihood tend to perform poorly for estimating the full joint $p(x)$, but can be usefull when only conditional distributions is needed.  
It can't be used with techniques giving a lower bound for $\log \tilde{p}(x)$, so it's difficult to use it for deep models.  

Pseudo likelihood need to compute all conditionals, but the cost can be reduced to the same than PCD if we compute one randomly selected conditional per example.

# Score Matching and Ratio Matching

The score is $\nabla_x \log p(x)$. Score matching minimize the following criterion:
$$L(x, \theta) = \frac{1}{2} || \nabla_x \log p_\text{model}(x;\theta) - \nabla_x \log p_\text{data}(x)||^2$$
$$J(\theta) = \frac{1}{2} \mathbb{E}_{p_\text{data}} L(x; \theta)$$

Miniziming $L(x, \theta)$ is equivalent to minizing the expectation of:

$$\tilde{L}(x, \theta) = \sum_{j=1}^n \left( \frac{\partial^2}{\partial x_j^2} \log p_\text{model}(x;\theta) + \frac{1}{2} \left( \frac{\partial}{\partial x_j} \log p_\text{model}(x;\theta) \right) ^2 \right)$$

We can't use lower bound on $\log \tilde{p}(x)$ because it gives no information on its derivatives, so it can't be used for deep models.  

Another approach, for binary data, is ratio matching. We minimize the expectation over the data of:
$$L^{(\text{RM})}(x; \theta) = \sum_{j=1}^n \left( \frac{1}{1 + \frac{p_\text{model}(x;\theta)}{p_\text{model}(f(x,j);\theta)}} \right) ^2$$

With $f(x,j)$ return $x$ with the bit at position $j$ flipped.  

Ratio matching can be useful for high-dimensional sparse data, like word-count vectors.

# Denoising Score Matching

Denoising Score matching regularize score matching by fitting a smoothe rdistribution:
$$p_\text{smoothed}(x) = \int p_\text{data}(y) q(x|y)dy$$
with $q(x|y)$ a corruption process that add some noise to $y$.  

Any consistent estimator with enough capacity will make $p_\text{model}$ into a set of Dirac distributions centered on the training points. Smoothing alleviate this problem.

# Noise-Contrastive Estimation

Most methods do not estimate the $Z$, only the log of the gradient for CD.  
Noise-contrastive Estimation (NCE) has an explicit form:
$$\log p_\text{model}(x) = \log \tilde{p}_\text{model}(x;\theta) + c$$
with $c$, approximation of $-\log Z(\theta)$, a parameter to be learned.  

$\log p_\text{model}(x)$ may not correspond to a valid probability distribution, but will become closer to valid as the estimate of $c$ improves.  

We train a probabilistic binary classifier, in such a way that the maximum likelihood estimate is an asymptotically consistent estimator of the original problem.  

We define a noise distribution, $p_\text{noise}(x)$, easy to evaluate and sample from. We construc a new model between $x$ and $y$ (determine if we sample from model or noise:
$$p_\text{joint}(y=1) = \frac{1}{2}$$
$$p_\text{join}{x|y=1} = p_\text{model}(x)$$
$$p_\text{join}{x|y=0} = p_\text{noise}(x)$$  

We can define a similar joint distribution for the training data, $p_\text{train}(y)$.  
We can use standard supervised learning with maximum likelihood criterion to fit $p_\text{joint}$ to $p_\text{train}$:
$$\theta^*, c^* = \arg \max_{\theta,c} \mathbb{E}_{x, y \sim p_\text{train}} \log p_\text{joint}(y|x)$$  

NCE is most successful when applied to problem with few random variables.  
For a large number of variables, the classifier can reject noise by identifing just one variable with unlikely values, and training slow down. 
It doesn't work with just a lower bound on $\log \tilde{p}$, it can only gives us lower and upper bounds on $p_\text{joint}$.  

Self-constrastive estimation define a new noise distribution at each step. Noise samples are generated from the model, it can be interpreted as forcing the model to learn to distinguish reality from its own evolving belief.

# Estimating the Partition Function

Estimate $Z$ is needed to compute the normalized ikelihood of the data.  

Let's suppose we want to compare model $M_A: p_A(x;\theta_A) = \frac{1}{Z_A} \tilde{p}_A(x;\theta_A)$ with $M_B: p_B(x;\theta_B) = \frac{1}{Z_B} \tilde{p}_B(x;\theta_B)$  

By using a test set of m examples $x^{(i)}$, we can say thay model $M_A$ is better than $M_B$ if:
$$\sum_i \log p_A(x^{(i)};\theta_A) - \sum_i \log p_B(x^{(i)};\theta_B) > 0$$

We can simplify the notation to only need the ratio of the two partitions functions:
$$\sum_i \left( \log \frac{\tilde{p}_A(x^{(i)};\theta_A)}{\tilde{p}_B(x^{(i)};\theta_B)} \right) - m \log \frac{Z_A}{Z_B}$$

If we wanted to compute the probability of the test data on $M_A$ or $M_B$, we would need to compute $Z$. If we knew the ratio $r = \frac{Z_B}{Z_A}$ and any of the two, we could compute the other:
$$Z_B = rZ_A$$  

We can use Monte-Carlo to estimate the partition function.  
We use a proposal distribution $p_0(x) = \frac{1}{Z_0} \tilde{p}_0(x)$, with tractable sampling, compution of $Z_0$, and $\tilde{p}_0(x)$.  
Let $p_1(x) = \frac{1}{Z_1} \tilde{p}_1(x)$ the distribution we want to estimate the partition function.  
We have the following Monte Carlo estimator:
$$\hat{Z}_1 = \frac{Z_0}{K} \sum_{k=1}^K \frac{\tilde{p}_1(x^{(k)})}{\tilde{p}_0(x^{(k)})} \space x^{(k)} \sim p_0$$

We can also estimate the ratio $\frac{Z_1}{Z_0}$ with:
$$\frac{1}{K} \sum_{k=1}^K \frac{\tilde{p}_1(x^{(k)})}{\tilde{p}_0(x^{(k)})} \space x^{(k)} \sim p_0$$

If the distribution $p_0$ is close to $p_1$, this can be an effictive estimate. But it's difficult to find a close and tractable distribution.

$$\hat{\text{Var}}(\hat{Z}_1) = \frac{Z_0}{K^2} \sum_{k=1}^K \left( \frac{\tilde{p}_1(x^{(k)})}{\tilde{p}_0(x^{(k)})} - \hat{Z}_1 \right) ^2$$

This quantity is largest when there is significant deviation on the importance weights $\frac{\tilde{p}_1(x^{(k)})}{\tilde{p}_0(x^{(k)})}$

## Annealed Importance Sampling

When $D_\text{KL}(p_0||p_1)$ is large, AIS attemds to bride the gap using intermediate distributions: $p_{\eta_0}, p_{\eta_1}, \text{...}, p_{\eta_n}$ with $p_0$ and $p_n$ first and last distribution.  

For example to estimate $Z_1$ of a RBM, the sequence are RBMs with weights interpolating between $0$ and the learned weights.  
We cab write the ratio as:
$$\frac{Z_1}{Z_0} = \prod_{j=0}^{n-1} \frac{Z_{\eta_{j+1}}}{Z_{\eta_j}}$$

Providing $p_{\eta_j}$ and $p_{\eta_{j+1}}$ are close enough we can get an accurate estimate of their ratio, and therefore of $\frac{Z_1}{Z_0}$.  

One popular choice of $p_{\eta_j}$ is to use a weighted geometric average of $p_1$ and $p_0$, with $Z_0$ known:
$$p_{\eta_j} \propto p_1^{\eta_j} p_0^{1 - \eta_j}$$

To sample from the different $p_{\eta_j}$, we use MCMC methods. We start with samples from $p_0$, and we sequentially generate samples with the markov chain until we arrive at samples from $p_1$.  

We generate $K$ samples from each $p_{\eta_j}$, and then we derive the importance weights for each of the K sequence of samples:
$$w^{(k)} = \frac{\tilde{p}_{\eta_1}(x_{\eta_1}^{(k)})}{\tilde{p}_{0}(x_{\eta_1}^{(k)})} \frac{\tilde{p}_{\eta_2}(x_{\eta_2}^{(k)})}{\tilde{p}_{\eta_1}(x_{\eta_2}^{(k)})} \text{...} \frac{\tilde{p}_{1}(x_{1}^{(k)})}{\tilde{p}_{\eta_{n-1}}(x_{1}^{(k)})}$$

The estimate of the ratio is given by:
$$\frac{Z_1}{Z_0} \approx \frac{1}{K} \sum_{k=1}^K w^{(k)}$$

It's currently the most common way to estimate $Z$ for undirected models.

## Bridge Sampling

Samle sampling relies on a single $p_*$, called the bridge, to interpolate berwwen $p_0$ and $p_1$. We estimate the ratio as:
$$\frac{Z_1}{Z_0} \approx \sum_{k=1}^K \frac{\tilde{p}_{*}(x_{0}^{(k)})}{\tilde{p}_{0}(x_{0}^{(k)})} / \sum_{k=1}^K \frac{\tilde{p}_{*}(x_{1}^{(k)})}{\tilde{p}_{1}(x_{1}^{(k)})} $$

The optimal bride dristibution is:
$$p_*(x) \propto \frac{\tilde{p}_0(x)\tilde{p}_1(x)}{r \tilde{p}_0(x) + \tilde{p}_1(x)}$$

where $r = Z_1 / Z_0$.  

We can start we a corse estimate of $r$, and use the resulting bride distribution to iteratively refine our estimate of $r$.  


If $D_\text{KL}(p_0||p_1)$ is not too large, bridge sample is often more effective to AIS to estimate $Z$. But with large distance, we can use AIS with many intermediate distributions.  
Linked importance sampling use both techniques to improve the estimate of $Z$.  

AIS is expensive and only used after training. Other strategies tries to maintain an estimate of $Z$ during training.